In [ ]:
!pip install scikit-learn==1.5.2
!pip install scikit-learn==1.5.2
!pip install scikeras[tensorflow]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 102.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scikeras.wrappers import KerasClassifier
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold , train_test_split
from scipy.stats import randint, uniform
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score , make_scorer, f1_score,precision_score, recall_score
from sklearn.neural_network import MLPClassifier

In [ ]:
np.random.seed(10)

In [ ]:
X = pd.read_csv("/content/drive/MyDrive/ML/reg_features.csv")
y = pd.read_csv("/content/drive/MyDrive/ML/reg_labels.csv")

In [ ]:
def classify_bgl(value):
    if value < 140:
        return 0
    elif value <= 199:
        return 1
    else:
        return 2

y['BGL'] = y['BGL'].apply(classify_bgl)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y['BGL'],
    test_size=0.2,
    random_state=10,
    stratify=y['BGL']
)

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# convert scaled arrays back to DataFrames
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)


from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=10)
X_train_scaled, y_train = smote.fit_resample(X_train_scaled, y_train)

#boosting (with Gradient Boosting)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report
from scipy.stats import randint as sp_randint
from scipy.stats import uniform


# Gradient Boosting
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=10
)


# Hyperparameter space
param_dist = {
    'n_estimators': sp_randint(50, 300),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': sp_randint(2, 10),
    'min_samples_split': sp_randint(2, 10),
    'min_samples_leaf': sp_randint(1, 10),
    'subsample': uniform(0.6, 0.4),
    'max_features': ['sqrt', 'log2',None] # Removed 'auto' as it is not a valid option
}

# Randomized Search
random_search = RandomizedSearchCV(
    gb_model,
    param_distributions=param_dist,
    n_iter=30,
    scoring='f1_weighted',
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=10,
    error_score='raise'
)

random_search.fit(X_train_scaled, y_train)
best_gb_model = random_search.best_estimator_
y_pred_gb = best_gb_model.predict(X_test_scaled)

'''# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred_gb))
print("F1 Score:", f1_score(y_test, y_pred_gb, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_pred_gb))'''

Fitting 5 folds for each of 30 candidates, totalling 150 fits


'# Evaluation\nprint("Accuracy:", accuracy_score(y_test, y_pred_gb))\nprint("F1 Score:", f1_score(y_test, y_pred_gb, average=\'weighted\'))\nprint("Classification Report:\n", classification_report(y_test, y_pred_gb))'

In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Predictions on training and test sets
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate on Training Data
    print(" Training Performance")
    print("Accuracy:", accuracy_score(y_train, y_train_pred))
    print("F1 Score:", f1_score(y_train, y_train_pred, average='weighted'))
    print("Classification Report:\n", classification_report(y_train, y_train_pred))

    # Evaluate on Test Data
    print(" Test Performance")
    print("Accuracy:", accuracy_score(y_test, y_test_pred))
    print("F1 Score:", f1_score(y_test, y_test_pred, average='weighted'))
    print("Classification Report:\n", classification_report(y_test, y_test_pred))

In [ ]:
print(evaluate_model(best_gb_model, X_train_scaled, y_train, X_test_scaled, y_test))

 Training Performance
Accuracy: 1.0
F1 Score: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       256
           1       1.00      1.00      1.00       256
           2       1.00      1.00      1.00       256

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768

 Test Performance
Accuracy: 0.8
F1 Score: 0.8009597637504615
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.89      0.90        65
           1       0.68      0.71      0.70        21
           2       0.58      0.58      0.58        19

    accuracy                           0.80       105
   macro avg       0.72      0.73      0.73       105
weighted avg       0.80      0.80      0.80       105

None


#AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from scipy.stats import randint, uniform
import numpy as np

# Define parameter grid for AdaBoost
param_dist_ada = {
    'n_estimators': np.arange(50, 300, 50),
    'learning_rate': uniform(0.01, 1),
    'estimator__max_depth': randint(2, 10),
    'estimator__ccp_alpha': uniform(0.0, 0.1) # pruning parameter for DecisionTreeClassifier
}

# Create base estimator
base_tree_tune = DecisionTreeClassifier(class_weight='balanced', random_state=10)

# Create AdaBoost model for tuning
ada_tune = AdaBoostClassifier(estimator=base_tree_tune, random_state=10)

# Perform RandomizedSearchCV for AdaBoost
random_search_ada = RandomizedSearchCV(
    ada_tune,
    param_distributions=param_dist_ada,
    n_iter=30, # Increase n_iter for better exploration
    cv=5,
    scoring='f1_weighted', # Use f1_weighted as per previous models
    n_jobs=-1,
    random_state=10
)

random_search_ada.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = random_search_ada.best_params_
print("Best Params from RandomizedSearchCV:", best_params)

# Base estimator with pruning
base_tree = DecisionTreeClassifier(
    max_depth=best_params['estimator__max_depth'],
    ccp_alpha=best_params['estimator__ccp_alpha'],
    class_weight='balanced',
    random_state=10
)

# AdaBoost model
ada_model = AdaBoostClassifier(
    estimator=base_tree,
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    random_state=10
)

'''#bagging
bagged_boost_model = BaggingClassifier(
    estimator=ada_model,
    n_estimators=10,  # number of bagging iterations
    random_state=10,
    n_jobs=-1
)'''

# Step 3: Fit the model
ada_model.fit(X_train_scaled, y_train)

# Evaluate on test set
y_test_pred = ada_model.predict(X_test_scaled)

print(" Final Evaluation on Test Set:")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("F1 Score:", f1_score(y_test, y_test_pred, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_test_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Best Params from RandomizedSearchCV: {'estimator__ccp_alpha': np.float64(0.014217004760152696), 'estimator__max_depth': 7, 'learning_rate': np.float64(0.5939013656536759), 'n_estimators': np.int64(100)}


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


 Final Evaluation on Test Set:
Accuracy: 0.7904761904761904
F1 Score: 0.7904991184205398
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.89      0.90        65
           1       0.60      0.71      0.65        21
           2       0.62      0.53      0.57        19

    accuracy                           0.79       105
   macro avg       0.71      0.71      0.71       105
weighted avg       0.79      0.79      0.79       105



#XGBoost



In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [2, 3, 4, 5, 6, 7, 8],  # limit tree depth
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8],
    'reg_alpha': [0, 0.01, 0.1, 1],   # L1 regularization
    'reg_lambda': [0.1, 0.5, 1.0]        # L2 regularization
}

# Create XGBoost model
xgb_model = XGBClassifier()

grid = RandomizedSearchCV(xgb_model, param_grid, cv=3, n_iter=30, scoring='f1_weighted', n_jobs=-1)

# Fit model
grid.fit(X_train_scaled, y_train)
best_model = grid.best_estimator_


# Predict
y_val_pred = best_model.predict(X_test_scaled)
y_train_pred = best_model.predict(X_train_scaled)

In [ ]:
print(evaluate_model(best_model, X_train_scaled, y_train, X_test_scaled, y_test))

 Training Performance
Accuracy: 1.0
F1 Score: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       256
           1       1.00      1.00      1.00       256
           2       1.00      1.00      1.00       256

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768

 Test Performance
Accuracy: 0.819047619047619
F1 Score: 0.813691537220949
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94        65
           1       0.65      0.71      0.68        21
           2       0.60      0.47      0.53        19

    accuracy                           0.82       105
   macro avg       0.73      0.71      0.72       105
weighted avg       0.81      0.82      0.81       105

None


#voting classification

##soft voting

In [ ]:
from sklearn.ensemble import VotingClassifier

# Create soft voting ensemble
voting_clf = VotingClassifier(
    estimators=[
        ('gb', best_gb_model),
        ('ada', ada_model),
        ('xgb', best_model)
    ],
    voting='soft',
    weights=[0.35, 0.25, 0.4],
    n_jobs=-1
)

# Fit the voting classifier
voting_clf.fit(X_train_scaled, y_train)

# Predict
y_pred_voting = voting_clf.predict(X_test_scaled)
y_train_pred_voting = voting_clf.predict(X_train_scaled)

# Evaluation
print(" Voting Classifier Performance on Training Set:")
print("Accuracy:", accuracy_score(y_train, y_train_pred_voting))
print("F1 Score:", f1_score(y_train, y_train_pred_voting, average='weighted'))
print("Classification Report:\n", classification_report(y_train, y_train_pred_voting))

print(" Voting Classifier Performance on Test Set:")
print("Accuracy:", accuracy_score(y_test, y_pred_voting))
print("F1 Score:", f1_score(y_test, y_pred_voting, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_pred_voting))


 Voting Classifier Performance on Training Set:
Accuracy: 1.0
F1 Score: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       256
           1       1.00      1.00      1.00       256
           2       1.00      1.00      1.00       256

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768

 Voting Classifier Performance on Test Set:
Accuracy: 0.8380952380952381
F1 Score: 0.8337558181167205
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.95      0.93        65
           1       0.71      0.71      0.71        21
           2       0.69      0.58      0.63        19

    accuracy                           0.84       105
   macro avg       0.77      0.75      0.76       105
weighted avg       0.83      0.84      0.83       105



##Hard Voting

In [ ]:
from sklearn.ensemble import VotingClassifier

# Create hard voting ensemble
hard_voting_clf = VotingClassifier(
    estimators=[
        ('gb', best_gb_model),
        ('ada', ada_model),
        ('xgb', best_model)
    ],
    voting='hard',
    weights=[0.35, 0.25, 0.4],
    n_jobs=-1
)

# Fit the hard voting classifier
hard_voting_clf.fit(X_train_scaled, y_train)

# Predict
y_pred_hard = hard_voting_clf.predict(X_test_scaled)
y_train_pred_hard = hard_voting_clf.predict(X_train_scaled)

# Evaluation
print(" Hard Voting Classifier Performance on Training Set:")
print("Accuracy:", accuracy_score(y_train, y_train_pred_hard))
print("F1 Score:", f1_score(y_train, y_train_pred_hard, average='weighted'))
print("Classification Report:\n", classification_report(y_train, y_train_pred_hard))

print(" Hard Voting Classifier Performance on Test Set:")
print("Accuracy:", accuracy_score(y_test, y_pred_hard))
print("F1 Score:", f1_score(y_test, y_pred_hard, average='weighted'))
print("Classification Report:\n", classification_report(y_test, y_pred_hard))


 Hard Voting Classifier Performance on Training Set:
Accuracy: 1.0
F1 Score: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       256
           1       1.00      1.00      1.00       256
           2       1.00      1.00      1.00       256

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768

 Hard Voting Classifier Performance on Test Set:
Accuracy: 0.8095238095238095
F1 Score: 0.8095272560388839
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.91      0.91        65
           1       0.70      0.76      0.73        21
           2       0.56      0.53      0.54        19

    accuracy                           0.81       105
   macro avg       0.72      0.73      0.73       105
weighted avg       0.81      0.81      0.81       105

